In [18]:
import torch 
from torch import nn
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import skew
from scipy.stats import kurtosis
from prettytable import PrettyTable
import os

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Cuda Device Available")
    print("Name of the Cuda Device: ", torch.cuda.get_device_name())
    print("GPU Computational Capablity: ", torch.cuda.get_device_capability())

else:
    device = torch.device("cpu")
    
def BuildDataset():
    df = pd.read_csv("data/atp_matches_2015.csv")
    for i in range(2016, 2023):
        #print("data/atp_matches_" + str(i) + ".csv")
        newDf = pd.read_csv("data/atp_matches_" + str(i) + ".csv")
        df = pd.concat([df, newDf], join="inner", ignore_index = True)
        #print(df.tail())
    df.drop('winner_seed', inplace=True, axis=1)
    df.drop('winner_entry', inplace=True, axis=1)
    df.drop('loser_seed', inplace=True, axis=1)
    df.drop('loser_entry', inplace=True, axis=1)
    return df

def main():
    df = BuildDataset()
    print(df.tail())
    
#main()
df = BuildDataset()
df

Cuda Device Available
Name of the Cuda Device:  NVIDIA GeForce RTX 3060 Laptop GPU
GPU Computational Capablity:  (8, 6)


tourney_id                  tourney_name surface  draw_size tourney_level  tourney_date  match_num  winner_id         winner_name winner_hand  winner_ht winner_ioc  winner_age  loser_id        loser_name loser_hand  loser_ht loser_ioc  loser_age              score  best_of round  minutes  w_ace  w_df  w_svpt  w_1stIn  w_1stWon  w_2ndWon  w_SvGms  w_bpSaved  w_bpFaced  l_ace  l_df  l_svpt  l_1stIn  l_1stWon  l_2ndWon  l_SvGms  l_bpSaved  l_bpFaced  winner_rank  winner_rank_points  loser_rank  loser_rank_points
0                              2015-339                      Brisbane    Hard         28             A      20150104          1     105357        John Millman           R      183.0        AUS   25.560575    105733    Rhyne Williams          R       NaN       USA  23.791923            6-3 6-1        3   R32     65.0    6.0   2.0    44.0     24.0      19.0      14.0      8.0        1.0        1.0    3.0   4.0    50.0     31.0      20.0       5.0      8.0        1.0        5.0        153.0               328.0       220.0              221.0
1                              2015-339                      Brisbane    Hard         28             A      20150104          2     103813     Jarkko Nieminen           L      185.0        FIN   33.453799    106045       Denis Kudla          R     180.0       USA  22.384668        4-6 6-1 6-4        3   R32    104.0    4.0   0.0    92.0     59.0      39.0      17.0     14.0        4.0        7.0    6.0   1.0    83.0     50.0      26.0      19.0     13.0        3.0        8.0         73.0               689.0       123.0              440.0
2                              2015-339                      Brisbane    Hard         28             A      20150104          3     105902     James Duckworth           R      183.0        AUS   22.956879    104468      Gilles Simon          R     183.0       FRA  30.023272            6-2 6-2        3   R32     68.0    4.0   0.0    45.0     27.0      20.0      11.0      8.0        2.0        3.0    2.0   1.0    56.0     37.0      22.0       5.0      8.0       10.0       15.0        125.0               430.0        21.0             1730.0
3                              2015-339                      Brisbane    Hard         28             A      20150104          4     104871       Jeremy Chardy           R      188.0        FRA   27.895962    104979    Andrey Golubev          R     185.0       KAZ  27.457906            6-4 6-4        3   R32     69.0    7.0   1.0    53.0     39.0      31.0      11.0     10.0        0.0        0.0    9.0   2.0    57.0     38.0      30.0       8.0     10.0        1.0        3.0         31.0              1195.0        72.0              691.0
4                              2015-339                      Brisbane    Hard         28             A      20150104          5     105373       Martin Klizan           L      190.0        SVK   25.486653    103781     Jurgen Melzer          L     183.0       AUT  33.623546  6-7(5) 7-6(6) 6-1        3   R32    144.0    9.0   4.0   130.0     79.0      55.0      27.0     16.0        6.0        8.0    4.0   4.0    95.0     62.0      40.0      19.0     15.0        4.0        8.0         34.0              1094.0       110.0              505.0
...                                 ...                           ...     ...        ...           ...           ...        ...        ...                 ...         ...        ...        ...         ...       ...               ...        ...       ...       ...        ...                ...      ...   ...      ...    ...   ...     ...      ...       ...       ...      ...        ...        ...    ...   ...     ...      ...       ...       ...      ...        ...        ...          ...                 ...         ...                ...
19617  2022-M-DC-2022-WG2-PO-GRE-JAM-01  Davis Cup WG2 PO: GRE vs JAM    Clay          4             D      20220304          4     209362  Aristotelis Thanos           R        NaN        GRE   20.895277    209080   Blai